In [1]:
import pytorch_lightning as pl
from distributions import *
from dataloaders import *
from visualization import *
from models import *

In [2]:
import warnings
import logging

warnings.filterwarnings("ignore", message=".*ModelCheckpoint.*")
warnings.filterwarnings("ignore", message=".*val_dataloader.*")
warnings.filterwarnings("ignore", message=".*num_workers.*")

pl_logger = logging.getLogger("pytorch_lightning")
pl_logger.setLevel(logging.WARNING)  # or logging.ERROR


# MNIST Dataloader

In [3]:
num_classes = 10
train_loader, test_loader = get_dataloaders(
    dataset_name='mnist', batch_size = 5000, num_views=1,
    contrastive = False, size=28, with_augmentation = False, num_workers = 0,
    max_train_samples=5000 
)

In [4]:
test_loader = train_loader 

In [5]:
def run_model(model, plot_config, max_epochs= 60, check_val_every_n_epoch=60):
    trainer = pl.Trainer(
        max_epochs=max_epochs,
        accelerator='gpu' if torch.cuda.is_available() else 'cpu',
        devices=1,
        check_val_every_n_epoch=check_val_every_n_epoch,
        log_every_n_steps = 1,
        callbacks=[PlotLogger(config=plot_config)]
    )
    trainer.fit(model, train_loader, test_loader)
    return model

In [6]:
plot_config = PlotConfig(
        show_plots=True,
        selected_plots= ['neighborhood_dist', 'embeddings'])

# Dimensionality Reduction

## Parametric SNE

In [7]:
SNE_config = IConConfig(mapper = SimpleCNN(output_dim=2, input_key= "image", output_key= "embeddings"),
                        supervisory_distribution = Gaussian(sigma = 5, input_key= "image", mask_diagonal = True),
                        learned_distribution = Gaussian(sigma = 1, input_key= "embeddings", mask_diagonal = True),
                        lr= 1e-3,
                        )
SNE_model = IConModel(SNE_config)
SNE_model = run_model(SNE_model, plot_config, max_epochs= 20, check_val_every_n_epoch=1)

Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

Training: |                                               | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

...

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

## Parametric t-SNE

In [ ]:
tSNE_config = IConConfig(mapper = SimpleCNN(output_dim=2, input_key= "image", output_key= "embeddings"),
                         supervisory_distribution = Gaussian(sigma = 5, input_key= "image", mask_diagonal = True),
                         learned_distribution = StudentT(gamma = 1, input_key= "embeddings", mask_diagonal = True),
                         lr= 5e-3,
                        )
tSNE_model = IConModel(tSNE_config)
tSNE_model = run_model(tSNE_model, plot_config)

## Parametric t-SNE with K-Nearest Neighbors

In [14]:
#tSNE_config = IConConfig(mapper = SimpleCNN(output_dim=2, input_key= "image", output_key= "embeddings"),
#                         supervisory_distribution = UniformKNN(k = 30, input_key= "image", mask_diagonal = True),
#                         learned_distribution = StudentT(gamma = 1, input_key= "embeddings", mask_diagonal = True),
#                         lr= 1e-2,
#                        )
#tSNE_model = IConModel(tSNE_config)
tSNE_model.lr = 1e-3
tSNE_model = run_model(tSNE_model, plot_config)

Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

Training: |                                               | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

## Non-Parametric SNE

In [15]:
from sklearn.decomposition import PCA
features = gather_batch_tensors(train_loader, "image", "index", 5000)
pca = PCA(n_components=2)
components = pca.fit_transform(features.cpu().numpy())
init_weights = torch.tensor(components, dtype=torch.float32)


In [16]:
tSNE_config = IConConfig(mapper = LookUpTable(num_embeddings=5000, embedding_dim=2, 
                                              input_key ="index", output_key= "embeddings", 
                                              init_weights = init_weights),
                         supervisory_distribution = UniformKNN(k = 100, mask_diagonal = True, input_key ="image"),
                         learned_distribution = StudentT(gamma = 1, mask_diagonal = True, input_key ="embeddings"),
                         lr=1,
                        )
tSNE_model = IConModel(tSNE_config)
tSNE_model = run_model(tSNE_model, plot_config)

Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

Training: |                                               | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

# Contrastive Methods

## Supervised Contrastive Learning

In [17]:
supCon_config = IConConfig(mapper = SimpleCNN(output_dim=3, unit_sphere = True, input_key= "image", output_key= "embeddings"),
                           supervisory_distribution = Label(input_key= "label"),
                           learned_distribution = Gaussian(sigma = 0.5, input_key = "embeddings"),
                           accuracy_mode = 'regular',
                           num_classes = num_classes,
                           linear_probe = True,
                           lr = 1e-3,
                        )
model = IConModel(supCon_config)
model = run_model(model, plot_config)

Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

Training: |                                               | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

In [19]:
model = run_model(model, plot_config)

Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

Training: |                                               | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

## SimCLR

In [20]:
train_loader, test_loader = get_dataloaders(
    dataset_name='mnist', batch_size = 10000, num_views=2,
    contrastive = True, size=28, with_augmentation = True, num_workers = 0, max_train_samples=10000,
)

In [21]:
test_loader = train_loader

In [22]:
supCon_config = IConConfig(mapper = SimpleCNN(output_dim=3, unit_sphere = True, input_key= ["image", "image1"], output_key= ["embeddings", "embeddings1"]),
                           supervisory_distribution = Augmentation(block_size=1, input_key = "index"),
                           learned_distribution = Gaussian(sigma = 0.5, input_key = "embeddings", input_key2 = "embeddings1", metric="dot", mask_diagonal=True),
                           accuracy_mode = 'regular',
                           num_classes = num_classes,
                           linear_probe = True,
                           lr = 1e-3,
                        )
model = IConModel(supCon_config)
model = run_model(model, plot_config,  max_epochs= 600, check_val_every_n_epoch=50)

Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

Training: |                                               | 0/? [00:00<?, ?it/s]


KeyboardInterrupt



# Cross Entropy

In [8]:
plot_config = PlotConfig(
        show_plots=True,
        selected_plots= ['neighborhood_dist', 'probabilities_star'],
)

In [9]:
cnn = SimpleCNN(output_dim=10, input_key= "image", output_key = "embeddings")
onehot = OneHotEncoder(num_classes = 10, fixed= True, input_key = "label", output_key = "class-one-hot")
classification_config = IConConfig(
                       mapper = [cnn, onehot],
                       supervisory_distribution = Label(pairwise= False, num_classes = 10, mask_diagonal = False, input_key = "label"),
                       learned_distribution= Gaussian(sigma = 0.5, metric= 'dot', mask_diagonal = False, 
                                                      input_key = "embeddings", input_key2 = "class-one-hot"),
                       accuracy_mode = 'regular',
                       num_classes = num_classes,
                       linear_probe = False,
                       lr = 0.001
                      )
model = IConModel(classification_config)
model = run_model(model, plot_config)

Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

/anaconda/envs/py38_default/lib/python3.10/site-packages/pytorch_lightning/utilities/data.py:79: UserWarning:

Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 5000. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.

/datadrive/shaden/ICon/visualization/ploty_callback.py:211: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



Training: |                                               | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]